**Q1.  Flux Limiters : Shock Capturing Schemes**

**Total Variation Diminising Scheme :**
 A numerical scheme is said to be total variation diminishing scheme (TVD) if :
$$ TV(q^{n+1}) \leq TV(q^n)$$

Such a scheme will not develop oscillations near a jump, because a jump is a monotonically increasing or decreasing function and TVD scheme will not increase the TV and keep it identically constant.  

##Slope Limiters :
Non-linear tools to prevent overshoots

When a sharp change occurs, the solution gradient become very steep, which makes solution unstable. These error arises as numerical ethods fails to keep track of the solution and result in unstable solution. So in order to acheive stability we basically limit the slope $σ_i^n$ of the piecewise linear scheme to make sure that the resulting schemem is TVD.

##Flux Limiters :    
Flux limiters help prevent quantities from overflowing by controlling how much material can flow through a certain point. They make sure that the flow stays within a safe limit to keep things stable.

Slope limiters and flux limiters are similar in what they do - they control quantities to prevent problems. But they differ in where they are used. Slope limiters work inside grids, while flux limiters are used on the interface.

In [ ]:
#importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib tk

In [ ]:
# Defining parameters
nx = 200  # Number of grid points
x = np.linspace(-10, 10, nx)
dx = x[1] - x[0]  # Grid spacing

t = np.linspace(0, 25, 1000)
dt = t[1] - t[0]  # Time step

h = 3.0  # Height of the initial wave
b = 8.0  # Half-width of the initial wave
u0 = np.where((np.abs(x) <= b / 2), h, 0)  # Initial condition: top-hat function
lamda = 3.0  # Speed of advection (constant velocity)


## Donner cell
For donner cell , $\phi  = 0 $, The formula reduces to:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right]$$

$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i+\frac{1}{2}})q^n_i + (1 - \theta_{i+\frac{1}{2}})q^n_{i+1} \right]$$

where
$$ \theta_{i-\frac{1}{2}} \equiv \theta(u_{i-\frac{1}{2}}) =
\begin{cases}
+1 & \text{for } \lambda \geq 0 \\
-1 & \text{for } \lambda \leq 0
\end{cases} $$

Same for i+1/2 case



In [ ]:
# Plotting setup
fig, ax = plt.subplots()
line, = ax.plot(x, u0, color='magenta')  # Initial plot
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title(f"Donner cell, $\lambda$ = {lamda}")

# Defining a function to update the plot
u1 = u0.copy()  # Initial copy of the solution
def update(frame):
    u = u1.copy()  # Copy the solution to avoid modifying it while iterating
    for i in range(len(x)):
        if lamda > 0:
            theta = 1.0
        else:
            theta = -1.0

        # Calculating fluxes
        f_minus_half = (lamda / 2) * ((1 + theta) * u[(i - 1) % nx] + (1 - theta) * u[(i) % nx])
        f_plus_half = (lamda / 2) * ((1 + theta) * u[(i) % nx] + (1 - theta) * u[(i + 1) % nx])

        # Updating
        u1[i] = u[i] + (dt / dx) * (f_minus_half - f_plus_half)

    line.set_ydata(u1)  # Updating the plot with the new solution

    return line,

# Create the animation
animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)

plt.show() #showing plot

## Lax Wenderoff
For lax Wenderoff $\phi  = 1.0$. The formula becomes:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) (1) (q^n_i - q^n_{i-1}) $$
$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i+\frac{1}{2}})q^n_i + (1 - \theta_{i+\frac{1}{2}})q^n_{i+1} \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) (1) (q^n_{i+1} - q^n_i) $$

In [ ]:
# Create figure and axis objects for plotting
fig, ax = plt.subplots()
line, = ax.plot(x, u0, color='magenta')  # Initial plot
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title(f"Lax-Wendroff, $\lambda$ = {lamda}")

u1 = u0.copy()  #copy of the initial condition

# Defining a function to update the plot
def update(frame):
    u = u1.copy()  # Copy the solution to avoid modifying it while iterating
    for i in range(len(x)):
        if lamda > 0:
            theta = 1.0
        else:
            theta = -1.0

        # Calculating fluxes
        f_minus_half = (lamda / 2) * ((1 + theta) * u[(i - 1) % nx] + (1 - theta) * u[(i) % nx] + (abs(lamda / 2)) * (1 - abs(lamda * dt / dx)) * (u[i % nx] - u[(i - 1) % nx]))
        f_plus_half = (lamda / 2) * ((1 + theta) * u[(i) % nx] + (1 - theta) * u[(i + 1) % nx] + (abs(lamda / 2)) * (1 - abs(lamda * dt / dx))) * (u[(i + 1) % nx] - u[(i) % nx])

        # Updating
        u1[i] = u[i] + (dt / dx) * (f_minus_half - f_plus_half)

    # Updating the plot with the new solution
    line.set_ydata(u1)

    return line,

# Create the animation
animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)

plt.show()  #showing plot


## Beam Warming
$\phi = r$, formula:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) r^n_{i-\frac{1}{2}} (q^n_i - q^n_{i-1})
 $$
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) r^n_{i+\frac{1}{2}} (q^n_i - q^n_{i-1})
 $$

where
$$ r_{i-\frac{1}{2}} =
\begin{cases}
    \frac{q^n_{i-1} - q^n_{i-2}}{q^n_{i} - q^n_{i-1}} & \text{for } \lambda \geq 0 \\
    \frac{q^n_{i+1} - q^n_i}{q^n_{i} - q^n_{i-1}} & \text{for } \lambda \leq 0
\end{cases} $$

$$ r_{i+\frac{1}{2}} =
\begin{cases}
    \frac{q^n_{i} - q^n_{i-1}}{q^n_{i+1} - q^n_{i}} & \text{for } \lambda \geq 0 \\
    \frac{q^n_{i+2} - q^n_{i+1}}{q^n_{i+1} - q^n_{i}} & \text{for } \lambda \leq 0
\end{cases} $$



In [ ]:
# Create figure and axis objects for plotting
fig, ax = plt.subplots()
line, = ax.plot(x, u0,color = 'magenta')  # Initial plot
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title(f"Beam-Warming, $\lambda $ = {lamda} ")

u1 = u0.copy()   #copy of the initial condition

# Defining a function to update the plot
def update(frame):
    u = u1.copy()    # Copy the solution to avoid modifying it while iterating
    # Determining the value of theta based on the sign of the advection speed as given
    # Calculating r_minus_half and its denominator
    # Avoiding division by zero and calculating r_minus_half
    for i in range(len(x)):
        if lamda > 0:
         theta = 1.0
         r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
         r_minus_half = u[(i-1)%nx] - u[(i-2)%nx] / r_minus_half_d if r_minus_half_d!=0 else 0
        else:
           theta = -1.0
           r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
           r_minus_half = (u[(i+1)%nx] - u[(i)%nx]) / r_minus_half_d if r_minus_half_d!=0 else 0

        # Calculating fluxes
        f_minus_half =  (lamda/2) * ( (1 + theta) * u[(i - 1) % nx] + (1-theta) * u[(i) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx)) ) * (u[i % nx] - u[(i-1)% nx]) * r_minus_half
        f_plus_half = (lamda/2) * ( (1 + theta) * u[(i) % nx] + (1-theta) * u[(i+1) % nx] + (abs(lamda/2)) * (1- abs(lamda * dt/dx))) * (u[(i+1) % nx] - u[(i)%nx])

        u1[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)  # Updating
    line.set_ydata(u1)   # Updating the plot with the new solution

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)  # Creating the animation
plt.show()


## Fromm
$\phi = \frac{1}{2} (1+r) $. The formula changes to:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \frac{1}{2}(1 + r^n_{i-\frac{1}{2}}) (q^n_i - q^n_{i-1})
 $$
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \frac{1}{2}(1 + r^n_{i+\frac{1}{2}}) (q^n_i - q^n_{i-1})
 $$

In [ ]:
# Create figure and axis objects for plotting
fig, ax = plt.subplots()
line, = ax.plot(x, u0,color = 'magenta')  # Initial plot
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title(f"Fromm, $\lambda $ = {lamda} ")

u1 = u0.copy()   #copy of the initial condition

# Defining a function to update the plot
def update(frame):
    u = u1.copy()   # Copy the solution to avoid modifying it while iterating
    # Determining the value of theta based on the sign of the advection speed as given
    # Calculating r_minus_half and its denominator
    # Avoiding division by zero and calculating r_minus_half
    for i in range(len(x)):
        if lamda > 0:
         theta = 1.0
         r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
         r_minus_half = u[(i-1)%nx] - u[(i-2)%nx] / r_minus_half_d if r_minus_half_d!=0 else 0

         r_plus_half_d = u[(i+1)%nx] - u[(i)%nx]
         r_plus_half = (u[(i)%nx] - u[(i-1)%nx]) / r_plus_half_d if r_plus_half_d !=0 else 0
        else:
           theta = -1.0
           r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
           r_minus_half = (u[(i+1)%nx] - u[(i)%nx]) / r_minus_half_d if r_minus_half_d!=0 else 0

           r_plus_half_d= (u[(i+1)%nx] - u[(i)%nx])
           r_plus_half = (u[(i+2)%nx] - u[(i+1)%nx]) / r_plus_half_d

        # Calculating fluxes
        f_minus_half =   (lamda/2) * ( (1 + theta) * u[(i - 1) % nx] + (1-theta) * u[(i) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[i % nx] - u[(i-1)% nx]
                        ) * ((1 + r_minus_half) / 2)
        f_plus_half = (lamda/2) * ( (1 + theta) * u[(i) % nx] + (1-theta) * u[(i+1) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[(i+1) % nx] - u[(i)% nx]
                        ) * ((1 + r_plus_half) / 2)

        u1[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u1)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)  #creating animation
plt.show()


## Min mod
$\phi = minmod(1,r) $

In [ ]:
# Create figure and axis objects for plotting
fig, ax = plt.subplots()
line, = ax.plot(x, u0,color = 'magenta') # Initial plot
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title(f"Minmod, $\lambda $ = {lamda} ")

u1 = u0.copy()

# Defining a function to update the plot
def update(frame):
    u = u1.copy()   # Copy the solution to avoid modifying it while iterating
    # Determining the value of theta based on the sign of the advection speed as given
    # Calculating r_minus_half and its denominator
    # Avoiding division by zero and calculating r_minus_half
    for i in range(len(x)):
        if lamda > 0:
         theta = 1.0
         r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
         r_minus_half = u[(i-1)%nx] - u[(i-2)%nx] / r_minus_half_d if r_minus_half_d!=0 else 0

         r_plus_half_d = u[(i+1)%nx] - u[(i)%nx]
         r_plus_half = (u[(i)%nx] - u[(i-1)%nx]) / r_plus_half_d if r_plus_half_d !=0 else 0
        else:
           theta = -1.0
           r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
           r_minus_half = (u[(i+1)%nx] - u[(i)%nx]) / r_minus_half_d if r_minus_half_d!=0 else 0

           r_plus_half_d= (u[(i+1)%nx] - u[(i)%nx])
           r_plus_half = (u[(i+2)%nx] - u[(i+1)%nx]) / r_plus_half_d

        # Calculating fluxes
        f_minus_half =   (lamda/2) * ( (1 + theta) * u[(i - 1) % nx] + (1-theta) * u[(i) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[i % nx] - u[(i-1)% nx]) * min(1,abs(r_minus_half))
        f_plus_half = (lamda/2) * ( (1 + theta) * u[(i) % nx] + (1-theta) * u[(i+1) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[(i+1) % nx] - u[(i)% nx]) * min(1,abs(r_plus_half))

        u1[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u1)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)  #creating animation
plt.show()


## Super bee
$\phi = max(0, min(1,2r), min(2,r)) $


In [ ]:
# Create figure and axis objects for plotting
fig, ax = plt.subplots()
line, = ax.plot(x, u0,color = 'magenta')  # Initial plot
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title(f"Superbee, $\lambda $ = {lamda} ")

u1 = u0.copy()

# Defining a function to update the plot
def update(frame):
    u = u1.copy()   # Copy the solution to avoid modifying it while iterating
    # Determining the value of theta based on the sign of the advection speed as given
    # Calculating r_minus_half and its denominator
    # Avoiding division by zero and calculating r_minus_half
    for i in range(len(x)):
        if lamda > 0:
         theta = 1.0
         r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
         r_minus_half = u[(i-1)%nx] - u[(i-2)%nx] / r_minus_half_d if r_minus_half_d!=0 else 0

         r_plus_half_d = u[(i+1)%nx] - u[(i)%nx]
         r_plus_half = (u[(i)%nx] - u[(i-1)%nx]) / r_plus_half_d if r_plus_half_d !=0 else 0
        else:
           theta = -1.0
           r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
           r_minus_half = (u[(i+1)%nx] - u[(i)%nx]) / r_minus_half_d if r_minus_half_d!=0 else 0

           r_plus_half_d= (u[(i+1)%nx] - u[(i)%nx])
           r_plus_half = (u[(i+2)%nx] - u[(i+1)%nx]) / r_plus_half_d

        # Calculating fluxes
        f_minus_half =   (lamda/2) * ( (1 + theta) * u[(i - 1) % nx] + (1-theta) * u[(i) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[i % nx] - u[(i-1)% nx]) * max(0,min(1,2*r_minus_half),min(2,r_minus_half))
        f_plus_half = (lamda/2) * ( (1 + theta) * u[(i) % nx] + (1-theta) * u[(i+1) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[(i+1) % nx] - u[(i)% nx]) * min(1,abs(r_plus_half))

        u1[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u1)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)   #creating animation
plt.show()


## MC
$\phi = max(0, min((1 + r)/2, 2, 2r)) $

In [ ]:
# Create figure and axis objects for plottingfig, ax = plt.subplots()
fig, ax = plt.subplots()
line, = ax.plot(x, u0,color = 'magenta')  # Initial plot
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title(f"MC, $\lambda $ = {lamda} ")

u1 = u0.copy()

# Defining a function to update the plot
def update(frame):
    u = u1.copy()   # Copy the solution to avoid modifying it while iterating
    # Determining the value of theta based on the sign of the advection speed as given
    # Calculating r_minus_half and its denominator
    # Avoiding division by zero and calculating r_minus_half
    for i in range(len(x)):
        if lamda > 0:
         theta = 1.0
         r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
         r_minus_half = u[(i-1)%nx] - u[(i-2)%nx] / r_minus_half_d if r_minus_half_d!=0 else 0

         r_plus_half_d = u[(i+1)%nx] - u[(i)%nx]
         r_plus_half = (u[(i)%nx] - u[(i-1)%nx]) / r_plus_half_d if r_plus_half_d !=0 else 0
        else:
           theta = -1.0
           r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
           r_minus_half = (u[(i+1)%nx] - u[(i)%nx]) / r_minus_half_d if r_minus_half_d!=0 else 0

           r_plus_half_d= (u[(i+1)%nx] - u[(i)%nx])
           r_plus_half = (u[(i+2)%nx] - u[(i+1)%nx]) / r_plus_half_d

       # Calculating fluxes
        f_minus_half =   (lamda/2) * ( (1 + theta) * u[(i - 1) % nx] + (1-theta) * u[(i) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[i % nx] - u[(i-1)% nx]) * max(0,min((1 + r_minus_half)/2, 2, 2*r_minus_half))
        f_plus_half = (lamda/2) * ( (1 + theta) * u[(i) % nx] + (1-theta) * u[(i+1) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[(i+1) % nx] - u[(i)% nx]) * min(1,abs(r_plus_half))

        u1[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u1)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)    #creating animation
plt.show()


### Van leer
$\phi =\frac{r + |r|}{1+|r|}  $ The formula changes to:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \frac{r_{i-1/2} + |r_{i-1/2}|}{1+|r_{i-1/2}|} (q^n_i - q^n_{i-1})
 $$
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \frac{r_{i+1/2} + |r_{i+1/2}|}{1+|r_{i+1/2}|} (q^n_i - q^n_{i-1})
 $$

In [ ]:
# Create figure and axis objects for plottingfig, ax = plt.subplots()
fig, ax = plt.subplots()
line, = ax.plot(x, u0,color = 'magenta') # Initial plot
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.set_title(f"van Leer, $\lambda $ = {lamda} ")

u1 = u0.copy()

# Defining a function to update the plot
def update(frame):
    u = u1.copy()   # Copy the solution to avoid modifying it while iterating
    # Determining the value of theta based on the sign of the advection speed as given
    # Calculating r_minus_half and its denominator
    # Avoiding division by zero and calculating r_minus_half
    for i in range(len(x)):
        if lamda > 0:
         theta = 1.0
         r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
         r_minus_half = u[(i-1)%nx] - u[(i-2)%nx] / r_minus_half_d if r_minus_half_d!=0 else 0

         r_plus_half_d = u[(i+1)%nx] - u[(i)%nx]
         r_plus_half = (u[(i)%nx] - u[(i-1)%nx]) / r_plus_half_d if r_plus_half_d !=0 else 0
        else:
           theta = -1.0
           r_minus_half_d = u[(i)%nx] - u[(i-1)%nx]
           r_minus_half = (u[(i+1)%nx] - u[(i)%nx]) / r_minus_half_d if r_minus_half_d!=0 else 0

           r_plus_half_d= (u[(i+1)%nx] - u[(i)%nx])
           r_plus_half = (u[(i+2)%nx] - u[(i+1)%nx]) / r_plus_half_d

        # Calculating fluxes
        f_minus_half =   (lamda/2) * ( (1 + theta) * u[(i - 1) % nx] + (1-theta) * u[(i) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[i % nx] - u[(i-1)% nx]
                        ) * ( (r_minus_half + abs(r_minus_half)) / (1 + abs(r_minus_half)) )
        f_plus_half = (lamda/2) * ( (1 + theta) * u[(i) % nx] + (1-theta) * u[(i+1) % nx] ) + abs((lamda/2)) * (1 - abs((lamda * dt / dx))) * (u[(i+1) % nx] - u[(i)% nx]) * min(1,abs(r_plus_half))

        u1[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u1)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)    #creating animation
plt.show()


**Q2. 1D Isothermal Shock Tube**




#### 1D isothermal equations in their conservative forms
* **Conservation of Mass**:
   $\frac{\partial \rho}{\partial t} + \frac{\partial (\rho v)}{\partial x} = 0$

* **Conservation of Momentum**:
   $\frac{\partial (\rho v)}{\partial t} + \frac{\partial (\rho v^2)}{\partial x} = -\frac{\partial P}{\partial x}$

* **Conservation of energy:**
   $$\frac{\partial (\rho e_{total})}{\partial t} + \frac{\partial (\rho v e_{total})}{\partial x} = -\frac{\partial (Pv)}{\partial x}$$

c0 = 1, $Δx = 1$

CFL Condition :
$Δt = 0.5\frac{Δx}{c0}$

$Δt =0.5$

#### Continuity equation
$$\frac{\partial \rho}{\partial t} + \frac{\partial (\rho v)}{\partial x} = 0$$
Given, $$\rho(x, t=0) = 2\Theta(50-x) + 1$$
$$v(x, t=0) = 0$$
With closed (reflective) boundaries.<br>
Let $\rho=q_1$ and $\rho v = q_1v = f_1$  <br>


#### Momentum conservation equation
$$\frac{\partial q_1}{\partial t} + \frac{\partial f_1}{\partial x} = 0$$
$$q_{1,i+1}^n = q_{1,i}^n - \frac{\Delta t}{\Delta x}[f_{1,i+\frac{1}{2}}^n - f_{1,i-\frac{1}{2}}^n]$$
And, let $\rho v = q_2$ and $\rho v.v = q_2v = f_2$<br>
First we will take RHS of the momentum conservation equation zero solving it for advection and then update the solution while considering the source term where $P = \rho c_s^2$<br>
$$\frac{\partial q_2}{\partial t} + \frac{\partial f_2}{\partial x} = 0$$
$$q_{2,i+1}^{n+\frac{1}{2}} = q_{2,i}^{n+\frac{1}{2}} - \frac{\Delta t}{\Delta x}[f_{2,i+\frac{1}{2}}^n - f_{2,i-\frac{1}{2}}^n]$$
$$q_{2,i+1}^n = q_{2,i+1}^{n+\frac{1}{2}}  - \frac{c_s^2}{2\Delta x}[q_{1,i+1}^n - q_{1,i-1}^n]$$

In [ ]:
x = np.linspace(0,100,101, endpoint=True) # Generating a linearly spaced array from 0 to 100 with 101 points
x_mid= (x[:-1]+x[1:])/2  # Calculating midpoints of the array
dx_mid = x_mid[1] - x_mid[0]   # Calculating the spacing between midpoints

# Defining a pulse function that returns 1 where x > 0 and 0 otherwise
def pulse(x):
  return np.where(x>0,1,0)

# Defining a function to determine the sign of a value
def ff(c):
  if c>=0:
    return 1
  else:
    return -1

# Initializing arrays to store state variables and related quantities
s = np.zeros([3,len(x_mid)])
t = np.zeros([2,len(x_mid)])

Cs= np.ones(len(x_mid)) # Assigning a constant value to Cs for all midpoints
time = 0.0
CFL = 0.5

# Setting initial values
s[0] = 2 * pulse(50 - x_mid) + 1
s[1] = 0
s[2] = s[0] * (Cs ** 2)
t[0] = s[0]
t[1] = s[0]*s[1]

density = []
velocity = []

# Main simulation loop running until time reaches 30
while time <= 30:
   # Storing current values of state variables
    t_upd = t
    s_upd = s
    t_temp = t
    # Initializing arrays for intermediate values
    v_intf = np.zeros(len(x))
    flux_intf = np.zeros([2, len(x)])
    a_1 = np.zeros(len(x_mid))

    # Calculating wave speeds at cell intfs
    for i in range(len(x_mid) - 1):
        a = max(abs(s_upd[1, i] - Cs[i]), abs(s_upd[1, i] + Cs[i]))
        a_1[i] = a

    # Calculating time step based on CFL condition
    dt_2 = CFL * dx_mid / max(a_1)

    # Calculating flux using Lax-Wendroff Flux Limiter
    for i in range(1, len(x) - 1):
        v_intf[i] = 0.5 * ((t_upd[1, i - 1] / t_upd[0, i - 1])+ (t_upd[1, i] / t_upd[0, i]))
        theta = ff(v_intf[i])
        # Lax-Wendroff flux limiter
        flux_intf[0, i] = 0.5 * v_intf[i] * ((1 + theta) * t_upd[0, i - 1] + (1 - theta) * t_upd[0, i]) - 0.5 * (dt_2 / dx_mid) * v_intf[i] ** 2 * ( t_upd[0, i] - t_upd[0, i - 1])
        flux_intf[1, i] = 0.5 * v_intf[i] * ((1 + theta) * t_upd[1, i - 1] + (1 - theta) * t_upd[1, i]) - 0.5 * (dt_2 / dx_mid) * v_intf[i] ** 2 * (t_upd[1, i] - t_upd[1, i - 1])


    # Updating conservative variables
    for i in range(1, len(x_mid) - 1):
        t_temp[0, i] = t_upd[0, i] - ((dt_2 / dx_mid) * (flux_intf[0, i + 1]- flux_intf[0, i]))  #rho update
        t_temp[1, i] = t_upd[1, i] - ((dt_2 / dx_mid) * (flux_intf[1, i + 1]- flux_intf[1, i]))   #flux update
        t_upd[0, i] = t_temp[0, i]

    # Updating velocity using
    for i in range(1, len(x_mid) - 1):
        t_upd[1, i] = t_temp[1, i] - 0.5 * ((Cs[i] ** 2) * (t_temp[0, i + 1] - t_temp[0, i - 1]) / dx_mid)

    # Using reflective boundary conditions
    t_upd[0, 0] = t_upd[0, 1]
    t_upd[0, -1] = t_upd[0, -2]
    t_upd[1, 0] = -t_upd[1, 1]
    t_upd[1, -1] = -t_upd[1, -2]

    # Updating state variables
    t = t_upd
    s[0] = t[0]
    s[1] = t[1] / t[0]
    s[2] = t[0] * (Cs ** 2)

    # Updating time
    time += dt_2

    # Adding updated values to the solution array
    density.append(list(s[0]))
    velocity.append(list(s[1]))

# Defininga a plotter function for 1D Isothermal Shock
def advection_plot(sol, heading):
    x = np.arange(0,100,1)
    fig = plt.figure()
    lines = plt.plot([])
    line = lines[0]
    plt.xlim(0,100)
    plt.ylim( min(sol[0])-0.1, max(sol[0])+0.2)
    plt.ylabel("Amplitude")
    plt.xlabel("Position(x)")
    plt.title(heading)
    def animate(frame):
        y = sol[frame]
        line.set_data((x,y))
    anim = FuncAnimation(fig, animate, frames=len(sol), interval=30)
    video = anim.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.close()


In [ ]:
# calling function to plot the evolution of density over time
advection_plot(density,"1D Isothermal Shock Tube (Density Evolution)")

In [ ]:
#calling function to plot the velocity over timee
advection_plot(velocity,"1D Isothermal Shock Tube (Velocity Evolution)")